In [1]:
import c3.libraries.fidelities as fidelities
from utils import *
import c3.libraries.algorithms as algorithms
from SingleQubitExperiment import SingleQubitExperiment

2021-09-03 09:52:59.320926: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-03 09:52:59.320962: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

def test_fidelity(propagators: dict, instructions: dict, index, dims,
                  qubit: chip.Qubit, generator: Generator, drive: chip.Drive, n_eval=-1):
    numPeaks = 2
    # find energies of the qubit
    energies = qubit.get_Hamiltonian().numpy().diagonal().real
    vec1 = np.array([energies[3] - energies[2], energies[1] - energies[0]]) / (2 * np.pi)

    infids = []
    for gate, propagator in propagators.items():
        # create signal and find peaks
        instr = instructions[gate]
        signal = generator.generate_signals(instr)[drive.name]
        peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), numPeaks)

        # add frequency offsets to infidelity
        #infid = np.linalg.norm(vec1 - peakFrequencies)
        infid1 = np.abs(vec1[0] - peakFrequencies[0]) / peakFrequencies[0]
        infid1 += np.abs(vec1[1] - peakFrequencies[1]) / peakFrequencies[1]

        # add peak height difference to infidelity
        if peakFrequencies[1] > peakFrequencies[0]:
            a = peakValues[1]
            b = np.sqrt(2) * peakValues[0]
        else:
            a = peakValues[0]
            b = np.sqrt(2) * peakValues[1]
        infid2 = np.abs(a - b) / np.maximum(a, b)
        infid = infid1 * (1 + infid2)
        print("peaks: ", np.sort(peakFrequencies), " => ", vec1, "; infidelity: ", infid1, infid2, infid)
        infids.append(infid)
    return tf.reduce_mean(infids)


def printInformation(exper: SingleQubitExperiment, name: str) -> None:
    signal = exper.generateSignal()
    peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), 4)
    print("peaks: ", np.sort(peakFrequencies))
    exper.plotSignal("signal_" + name)
    exper.plotTimeEvolution("populations_" + name)


In [3]:
freq = 5e9
anharm = 330e6
t_final = 7e-9
sigma = t_final / 4
#sigma2 = sigma
#relative_amp = np.sqrt(2)
carrier_freq = 5e9
optimisable_params = {
    "gauss": ["amp", #"sigma",
              "xy_angle", "freq_offset",  # for single gaussian envelope
              #"t_final", "t_bin_start", "t_bin_end",
              #"sigma2", "relative_amp",  # for double-gaussian envelope
              "delta",  # for DRAG
              "inphase",
              ],
    "carrier": [
        "freq",
        "framechange"
    ]
}

#amp = 2.9898805422620756 #V
#sigma = 7.914863107804569e-10 #s
#sigma2 = 6.955276846125239e-10 #s
#relative_amp = 1.8131542761817843 #
#delta = 0.004339356027326602 #
#carrier_freq = 4655811329.747005 #Hz 2pi
#framechange = 2.55339714228929 #rad

for i in range(3):
    envelope = createPWCDoubleGaussianPulse(t_final, sigma, 30)
    exper = SingleQubitExperiment(directory=f"./output/c_pwcG_{i}")
    exper.prepare(t_final, freq, anharm, carrier_freq, envelope)

    # print information
    energies = exper.getEnergies()
    print("energies: ", [(energies[i + 1] - energies[i]) / (2 * np.pi) for i in range(len(energies) - 1)])
    printInformation(exper, "before")

    params_after = exper.optimise(
        optimisable_params,
        algorithm=algorithms.cmaes,
        algorithm_options={"stop_at_convergence": 30},
        fidelity_function=fidelities.unitary_infid_set,
        fidelity_params={
            #'psi_0': exper.getInitialState()[:4],
            "active_levels": 4,
            # 'num_gates': len(gates)
            # "qubit": q1,
            # "generator": generator,
            # "drive": getDrive(model, q1)
        },
    )
    printInformation(exper, "after")
    print(params_after)

2021-09-03 09:54:58.055449: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-03 09:54:58.055742: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-03 09:54:58.055762: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-03 09:54:58.055796: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-09-03 09:54:58.056425: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-03 09:54:59.096990: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-03 09:5

energies:  [5000000000.0, 4670000000.000003, 4339999999.999994, 4010000000.0000033]
peaks:  [3.30165083e+09 4.40220110e+09 5.10255128e+09 6.20310155e+09]
saving plot in ./output/test/signal_before_None.png
saving plot in ./output/test/populations_before.png
C3:STATUS:Saving as: /home/user/c3/output/test/log_None_10.00/c1_unitary_infid_set_cmaes/2021_09_03_T_09_55_01/open_loop.log
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 7 (seed=768255, Fri Sep  3 09:55:01 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 7.098985294497424e-01 1.0e+00 8.60e-02  8e-02  9e-02 0:06.4
    2     18 4.459025994718221e-01 1.2e+00 9.33e-02  9e-02  1e-01 0:12.7
    3     27 5.743817724068945e-01 1.5e+00 1.03e-01  9e-02  1e-01 0:19.1
before:
 lower-X[0]-d1-gauss-amp               : 3.000 V 
lower-X[0]-d1-gauss-sigma             : 666.667 ps 
lower-X[0]-d1-gauss-sigma2            : 666.667 ps 
lower-X[0]-d1-gauss-relative_amp      : 1.414  
lower-X[0]-d1-gauss-delta 